# Portfolio Risk Dashboard

The purpose of this project is to practice creating a dashboard to easily observe and take note of portfolio risk.
Since the last year has been affected by COVID-19 we are only going to pull pricing data for risk calculation from the last 6 months.

### Importing Packages

In [24]:
import numpy as np
import pandas as pd
import requests

### Importing Data

For this project, we will be using the financial information publicly available from Voss Capital's 2021 13F. The data is publicly available at this address: https://whalewisdom.com/filer/voss-capital-llc#tabholdings_tab_link
The data available on a 13F is "[b]asically anything that is traded on an exchange as well as certain equity options and warrants, shares of closed-end investment companies and certain convertible debt securities." Source: whalewisdom.com

### About the Data in the 13F

The data tells us only what quarter the shares were purchased in and the predicted price that the fund paid for the amount. 

### About Stock Vantage API

Can provide stock data

import a mock csv of what we are looking for.

In [7]:
df = pd.read_csv("data/portfolio_csv_input.csv")

Figuring out how to index through these. 

In [9]:
df.head()

,ticker,shares,buy_price,buy_date
0,TSLA,2,4.98,4-3-21
1,DIS,2,2342.00,5-7-21
2,KO,4,324.00,2-2-21
3,BRK.B,2,1112.00,8-9-21


Let's take a look at the data types

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     4 non-null      object 
 1   shares     4 non-null      int64  
 2   buy_price  4 non-null      float64
 3   buy_date   4 non-null      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 256.0+ bytes


lets practice indexing a dataframe

In [20]:
for i in df.ticker:
    print(i)

TSLA
DIS
KO
BRK.B


In [22]:
df.head()

,ticker,shares,buy_price,buy_date
0,TSLA,2,4.98,4-3-21
1,DIS,2,2342.00,5-7-21
2,KO,4,324.00,2-2-21
3,BRK.B,2,1112.00,8-9-21


### Now let's see if we can get some historical data using the Alpha Vantage API.

In [190]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TSLA&apikey=0HG27IzWW0AWYH0X'
r = requests.get(url)
data = r.json()
print(data)

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'TSLA', '3. Last Refreshed': '2022-04-20', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2022-04-20': {'1. open': '1030.0000', '2. high': '1034.0000', '3. low': '975.2501', '4. close': '977.2000', '5. volume': '21616141'}, '2022-04-19': {'1. open': '1005.0600', '2. high': '1034.9400', '3. low': '995.3250', '4. close': '1028.1500', '5. volume': '16615944'}, '2022-04-18': {'1. open': '989.0300', '2. high': '1014.9200', '3. low': '973.4100', '4. close': '1004.2900', '5. volume': '17238407'}, '2022-04-14': {'1. open': '999.2900', '2. high': '1012.7099', '3. low': '982.1900', '4. close': '985.0000', '5. volume': '19474135'}, '2022-04-13': {'1. open': '981.0750', '2. high': '1026.2400', '3. low': '973.0970', '4. close': '1022.3700', '5. volume': '18373737'}, '2022-04-12': {'1. open': '997.6400', '2. high': '1021.1921', '3. low': '976.6001', '4. close': '98

The type of this data is a dictionary.

In [119]:
type(data)

dict

so, how can we iterate through this dictionary to get what we need? We need to find all of the attributes with 'close' as the key.

In [195]:
#Exploring the methods and attributes

dir(data)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In the previous code, one of the attributes is '_iter_' which is is a method that is called when an iterator is required for a container, and it should return a new iterator object that can iterate through all the objects in the container. 

For mappings (like dictionaries), .__iter__() should iterate over the keys. This means that if you put a dictionary directly into a for loop, Python will automatically call .__iter__() on that dictionary, and you’ll get an iterator over its keys.

Source: https://realpython.com/iterate-through-dictionary-python/

In [203]:
data

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes',
  '2. Symbol': 'TSLA',
  '3. Last Refreshed': '2022-04-20',
  '4. Output Size': 'Compact',
  '5. Time Zone': 'US/Eastern'},
 'Time Series (Daily)': {'2022-04-20': {'1. open': '1030.0000',
   '2. high': '1034.0000',
   '3. low': '975.2501',
   '4. close': '977.2000',
   '5. volume': '21616141'},
  '2022-04-19': {'1. open': '1005.0600',
   '2. high': '1034.9400',
   '3. low': '995.3250',
   '4. close': '1028.1500',
   '5. volume': '16615944'},
  '2022-04-18': {'1. open': '989.0300',
   '2. high': '1014.9200',
   '3. low': '973.4100',
   '4. close': '1004.2900',
   '5. volume': '17238407'},
  '2022-04-14': {'1. open': '999.2900',
   '2. high': '1012.7099',
   '3. low': '982.1900',
   '4. close': '985.0000',
   '5. volume': '19474135'},
  '2022-04-13': {'1. open': '981.0750',
   '2. high': '1026.2400',
   '3. low': '973.0970',
   '4. close': '1022.3700',
   '5. volume': '18373737'},
  '2022-04-12': {'1. o

### Now we can get the dates from the dictionary.

In [241]:
data = data['Time Series (Daily)']

In [245]:
dates = data.keys()

In [246]:
dates

dict_keys(['2022-04-20', '2022-04-19', '2022-04-18', '2022-04-14', '2022-04-13', '2022-04-12', '2022-04-11', '2022-04-08', '2022-04-07', '2022-04-06', '2022-04-05', '2022-04-04', '2022-04-01', '2022-03-31', '2022-03-30', '2022-03-29', '2022-03-28', '2022-03-25', '2022-03-24', '2022-03-23', '2022-03-22', '2022-03-21', '2022-03-18', '2022-03-17', '2022-03-16', '2022-03-15', '2022-03-14', '2022-03-11', '2022-03-10', '2022-03-09', '2022-03-08', '2022-03-07', '2022-03-04', '2022-03-03', '2022-03-02', '2022-03-01', '2022-02-28', '2022-02-25', '2022-02-24', '2022-02-23', '2022-02-22', '2022-02-18', '2022-02-17', '2022-02-16', '2022-02-15', '2022-02-14', '2022-02-11', '2022-02-10', '2022-02-09', '2022-02-08', '2022-02-07', '2022-02-04', '2022-02-03', '2022-02-02', '2022-02-01', '2022-01-31', '2022-01-28', '2022-01-27', '2022-01-26', '2022-01-25', '2022-01-24', '2022-01-21', '2022-01-20', '2022-01-19', '2022-01-18', '2022-01-14', '2022-01-13', '2022-01-12', '2022-01-11', '2022-01-10', '2022-01-

In [248]:
data

{'2022-04-20': {'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 '2022-04-19': {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 '2022-04-18': {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 '2022-04-14': {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 '2022-04-13': {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 '2022-04-12': {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 '2022-04-11': {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '197857

### Now we have to figure out how to pull out the closing stock prices from the data

In [253]:
data

{'2022-04-20': {'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 '2022-04-19': {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 '2022-04-18': {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 '2022-04-14': {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 '2022-04-13': {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 '2022-04-12': {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 '2022-04-11': {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '197857

In [268]:
for key, value in data.items():
    print(key , value)


2022-04-20 {'1. open': '1030.0000', '2. high': '1034.0000', '3. low': '975.2501', '4. close': '977.2000', '5. volume': '21616141'}
2022-04-19 {'1. open': '1005.0600', '2. high': '1034.9400', '3. low': '995.3250', '4. close': '1028.1500', '5. volume': '16615944'}
2022-04-18 {'1. open': '989.0300', '2. high': '1014.9200', '3. low': '973.4100', '4. close': '1004.2900', '5. volume': '17238407'}
2022-04-14 {'1. open': '999.2900', '2. high': '1012.7099', '3. low': '982.1900', '4. close': '985.0000', '5. volume': '19474135'}
2022-04-13 {'1. open': '981.0750', '2. high': '1026.2400', '3. low': '973.0970', '4. close': '1022.3700', '5. volume': '18373737'}
2022-04-12 {'1. open': '997.6400', '2. high': '1021.1921', '3. low': '976.6001', '4. close': '986.9500', '5. volume': '21992032'}
2022-04-11 {'1. open': '980.4000', '2. high': '1008.4681', '3. low': '974.6400', '4. close': '975.9300', '5. volume': '19785735'}
2022-04-08 {'1. open': '1043.2100', '2. high': '1048.4400', '3. low': '1022.4350', '4

In [271]:
prices = []
for value in data.values():
    prices.append(value)

In [272]:
prices

[{'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '19785735'},
 {'1. open': '1043.2100',
  '2. high': '1048.4400',
  '3. low': '1022.4350',
  '4. close': '

### Now that we have isolated the prices, we need to put them into dataframes and join them.

It is a list and we need to turn it into a dataframe.

In [273]:
type(prices)

list

In [301]:
stock_df = pd.DataFrame(prices)
stock_df.head()

,1. open,2. high,3. low,4. close,5. volume
0,1030.0000,1034.0000,975.2501,977.2000,21616141
1,1005.0600,1034.9400,995.3250,1028.1500,16615944
2,989.0300,1014.9200,973.4100,1004.2900,17238407
3,999.2900,1012.7099,982.1900,985.0000,19474135
4,981.0750,1026.2400,973.0970,1022.3700,18373737


In [317]:
stock_df = stock_df.rename(columns = {'4. close':'close'})

In [318]:
dates_df = pd.DataFrame(dates)
dates_df.head()

,0
0,2022-04-20
1,2022-04-19
2,2022-04-18
3,2022-04-14
4,2022-04-13


### Now the DataFrames can be joined. 

In [321]:
whole_df = dates_df.join(stock_df)
whole_df = whole_df[[0,'close']]
whole_df = whole_df.rename(columns = {0:'date'})

In [322]:
whole_df.head()

,date,close
0,2022-04-20,977.2000
1,2022-04-19,1028.1500
2,2022-04-18,1004.2900
3,2022-04-14,985.0000
4,2022-04-13,1022.3700


In [ ]:
data

Here we see that this did not give us the result that we were looking for. These are the only keys. Let's see if we can try again and get more information. 

In [206]:
for key in data['Meta Data']:
    print(key)

1. Information
2. Symbol
3. Last Refreshed
4. Output Size
5. Time Zone


In [120]:
avdf = pd.DataFrame.from_dict(data)
avdf.head(20)
df2 = avdf[5:]
df2.head()


,Meta Data,Time Series (Daily)
2022-04-20,NaN,"{'1. open': '1030.0000', '2. high': '1034.0000..."
2022-04-19,NaN,"{'1. open': '1005.0600', '2. high': '1034.9400..."
2022-04-18,NaN,"{'1. open': '989.0300', '2. high': '1014.9200'..."
2022-04-14,NaN,"{'1. open': '999.2900', '2. high': '1012.7099'..."
2022-04-13,NaN,"{'1. open': '981.0750', '2. high': '1026.2400'..."


In [121]:
df2['New'] = ''

/Users/jaklynsoler/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
df2.head()

,Meta Data,Time Series (Daily),New
2022-04-20,NaN,"{'1. open': '1030.0000', '2. high': '1034.0000...",
2022-04-19,NaN,"{'1. open': '1005.0600', '2. high': '1034.9400...",
2022-04-18,NaN,"{'1. open': '989.0300', '2. high': '1014.9200'...",
2022-04-14,NaN,"{'1. open': '999.2900', '2. high': '1012.7099'...",
2022-04-13,NaN,"{'1. open': '981.0750', '2. high': '1026.2400'...",


In [123]:
df2.New = df2['Time Series (Daily)']

/Users/jaklynsoler/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [144]:
df2.New = df2.New.astype("str")
df2.head()

/Users/jaklynsoler/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Meta Data,Time Series (Daily),New
2022-04-20,NaN,"{'1. open': '1030.0000', '2. high': '1034.0000...","{'1. open': '1030.0000', '2. high': '1034.0000..."
2022-04-19,NaN,"{'1. open': '1005.0600', '2. high': '1034.9400...","{'1. open': '1005.0600', '2. high': '1034.9400..."
2022-04-18,NaN,"{'1. open': '989.0300', '2. high': '1014.9200'...","{'1. open': '989.0300', '2. high': '1014.9200'..."
2022-04-14,NaN,"{'1. open': '999.2900', '2. high': '1012.7099'...","{'1. open': '999.2900', '2. high': '1012.7099'..."
2022-04-13,NaN,"{'1. open': '981.0750', '2. high': '1026.2400'...","{'1. open': '981.0750', '2. high': '1026.2400'..."


In [186]:
df2.New.str.slice_replace(start=0, stop=83, repl='')

2022-04-20                                          
2022-04-19     '1028.1500', '5. volume': '16615944'}
2022-04-18      1004.2900', '5. volume': '17238407'}
2022-04-14       985.0000', '5. volume': '19474135'}
2022-04-13      1022.3700', '5. volume': '18373737'}
                               ...                  
2021-12-02     '1084.6000', '5. volume': '24371623'}
2021-12-01     '1095.0000', '5. volume': '22934698'}
2021-11-30     '1144.7600', '5. volume': '27092038'}
2021-11-29     '1136.9900', '5. volume': '19464467'}
2021-11-26     '1081.9200', '5. volume': '11680890'}
Name: New, Length: 100, dtype: object

In [157]:
df2.head()

,Meta Data,Time Series (Daily),New
2022-04-20,NaN,"{'1. open': '1030.0000', '2. high': '1034.0000...",977.2000
2022-04-19,NaN,"{'1. open': '1005.0600', '2. high': '1034.9400...","{'1. open': '1005.0600', '2. high': '1034.9400..."
2022-04-18,NaN,"{'1. open': '989.0300', '2. high': '1014.9200'...","{'1. open': '989.0300', '2. high': '1014.9200'..."
2022-04-14,NaN,"{'1. open': '999.2900', '2. high': '1012.7099'...","{'1. open': '999.2900', '2. high': '1012.7099'..."
2022-04-13,NaN,"{'1. open': '981.0750', '2. high': '1026.2400'...","{'1. open': '981.0750', '2. high': '1026.2400'..."


In [125]:
type(df2.New)

pandas.core.series.Series

In [126]:
def sharpe_calc(array):

#Each array row has: ticker, num_shares, price_per_share, date_of_purchase

    for i in array[0]:
        print(i)



### Objective One: Create a function where the input is number of shares, purchase price, and purchase date. The function should return the Sharpe Ratio. 

To get the sharpe calculation of one stock:

we have to take the mean price difference per day for the last year starting at the date of purchase

BUT we want to have a cumulative sharpe ratio for all of the stocks that we own so we want the function to take in an array that is 4 columns: ticker, purchase price, purchase date and number of shares. 

Then we want the function to take the last year of the prices and find the differences between each price day by day and then find the average of that. 

Then we want the function to save that number to the average_daily_return list. 
We want the function to do the same thing with the 3 month Treasury rate and create a list with one number per array row. 

Then we want to take the sum of the average_daily_return list.
Then we want to take the sum of the 3mo_tbill list.

Then we want to take the avg_daily_return_sum minus the 3mo_Tbill_sum to find the numerator.
Then we want to take the standard deviation of the numerator to find the denominator. 
Then we want to divide the numerator by the denominator to find the sharpe ratio of the portfolio.